# Post Processing

In [ ]:
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
from coffea import hist, util
from coffea.processor import accumulate
from plots.helpers import makePlot2
from tools.helpers import cutflow_scale_and_merge, getCutFlowTable
from yahist import Hist1D, Hist2D

## Import Histograms

In [ ]:
scaled_output = accumulate(
        [
            util.load("/home/users/dspitzba/snowblower/analysis/outputs/output_signal_scaled_run20220125_150734.coffea"),
            util.load("/home/users/dspitzba/snowblower/analysis/outputs/output_QCD_scaled_run20220125_171824.coffea"),
            util.load("/home/users/dspitzba/snowblower/analysis/outputs/output_TT_scaled_run20220125_171025.coffea"),
            util.load("/home/users/dspitzba/snowblower/analysis/outputs/output_W_scaled_run20220125_170224.coffea"),
            util.load("/home/users/dspitzba/snowblower/analysis/outputs/output_Z_scaled_run20220125_183109.coffea"),
            #util.load("/home/users/dspitzba/snowblower/analysis/outputs/output_other_scaled_run20220125_163014.coffea"),
        ]
    )

In [ ]:
scaled_output = accumulate(
        [
            util.load("/home/users/ewallace/snowblower/analysis/outputs/output_all_scaled_run20220131_132242.coffea"),
        ]
    )

# Data Cards

In [ ]:
MT_vs_sdmass = {
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750':    {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250': {},
    'ZJetsToNuNu_HT':        {},
    'TT':        {},
    'WJetsToLNu':        {},
    'QCD_bEnriched_HT':        {},
    'other': {},
}

variations = [
    '',
    '_up',
    '_down',
    '_0b_up',
    '_0b_down',
    '_1b_up',
    '_1b_down',
    '_2b_up',
    '_2b_down',
    '_1h_up',
    '_1h_down',
]

JES_unc = {}

b0_unc = {}

b1_unc = {}

b2_unc = {}

h1_unc = {}

In [ ]:
for sample in MT_vs_sdmass:
    for var in variations:
        MT_vs_sdmass[sample][var] = Hist2D.from_bincounts(
            scaled_output['MT_vs_sdmass'+var].values(overflow='over')[(sample,)].T,
            (scaled_output['MT_vs_sdmass'+var].axis('mt').edges(overflow='over'), scaled_output['MT_vs_sdmass'+var].axis('mass').edges(overflow='over')),
            errors = np.sqrt(scaled_output['MT_vs_sdmass'+var].values(sumw2=True, overflow='over')[(sample,)][1].T),
        )
        for b in range(0,4):
            for j in range(0,10):
                if round(MT_vs_sdmass[sample][var].counts[b][j], 3) == 0:
                    MT_vs_sdmass[sample][var].counts[b][j] = 0.01
                if round(MT_vs_sdmass[sample][var].errors[b][j], 3) == 0:
                    MT_vs_sdmass[sample][var].errors[b][j] = 0.01
                    
    JES_unc[sample] = abs((MT_vs_sdmass[sample]['_up']-MT_vs_sdmass[sample]['_down']).counts)/(2*MT_vs_sdmass[sample][''].counts)+1
    b0_unc[sample]  = abs((MT_vs_sdmass[sample]['_0b_up']-MT_vs_sdmass[sample]['_0b_down']).counts)/(2*MT_vs_sdmass[sample][''].counts)+1
    b1_unc[sample]  = abs((MT_vs_sdmass[sample]['_1b_up']-MT_vs_sdmass[sample]['_1b_down']).counts)/(2*MT_vs_sdmass[sample][''].counts)+1
    b2_unc[sample]  = abs((MT_vs_sdmass[sample]['_2b_up']-MT_vs_sdmass[sample]['_2b_down']).counts)/(2*MT_vs_sdmass[sample][''].counts)+1
    h1_unc[sample]  = abs((MT_vs_sdmass[sample]['_1h_up']-MT_vs_sdmass[sample]['_1h_down']).counts)/(2*MT_vs_sdmass[sample][''].counts)+1

## datacards

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1500_150_2.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1250_750_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1500_750_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1600_750_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1750_750_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1900_750_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_2000_750_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_2250_750_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_750_MH4_250_MH2_750_MHC_750'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_750_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1000_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1250_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1500_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1600_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1750_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_1900_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_2000_250_4.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('other_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['other', 'Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][''],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT'][''],
            'Wjets': MT_vs_sdmass['WJetsToLNu'][''],
            'tt': MT_vs_sdmass['TT'][''],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT'][''],
            'other': MT_vs_sdmass['other'][''],
        }
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'other',    round(MT_vs_sdmass['other'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT'][''].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        c.specifyUncertainty('other_unc',   binname, 'other',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'other',  JES_unc['other'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'other',  b0_unc['other'][b][j])
       
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'other',  b1_unc['other'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'other',  b2_unc['other'][b][j])
       
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT'][''].counts[b][j] + MT_vs_sdmass['WJetsToLNu'][''].counts[b][j] + MT_vs_sdmass['TT'][''].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts[b][j]++ MT_vs_sdmass['other'][''].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('Data_Cards/2HDMa_2250_250_4.txt')

## Plotting Regions

In [ ]:
bin_array_signal = np.ones(40)
i=0
for row in MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][''].counts:
    for col in row:
        bin_array_signal[i] = col
        i+=1

In [ ]:
bin_array_signal_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1000_MH4_250_MH2_1000_MHC_1000'][''].errors:
    for col in row:
        bin_array_signal_errors[i] = col
        i+=1

In [ ]:
bin_array_Z = np.ones(40)
i=0
for row in MT_vs_sdmass['ZJetsToNuNu_HT'][''].counts:
    for col in row:
        bin_array_Z[i] = col
        i+=1

In [ ]:
bin_array_Z_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['ZJetsToNuNu_HT'][''].errors:
    for col in row:
        bin_array_Z_errors[i] = col
        i+=1

In [ ]:
bin_array_tt = np.ones(40)
i=0
for row in MT_vs_sdmass['TT'][''].counts:
    for col in row:
        bin_array_tt[i] = col
        i+=1

In [ ]:
bin_array_tt_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['TT'][''].errors:
    for col in row:
        bin_array_tt_errors[i] = col
        i+=1

In [ ]:
bin_array_W = np.ones(40)
i=0
for row in MT_vs_sdmass['WJetsToLNu'][''].counts:
    for col in row:
        bin_array_W[i] = col
        i+=1

In [ ]:
bin_array_W_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['WJetsToLNu'][''].errors:
    for col in row:
        bin_array_W_errors[i] = col
        i+=1

In [ ]:
bin_array_QCD = np.ones(40)
i=0
for row in MT_vs_sdmass['QCD_bEnriched_HT'][''].counts:
    for col in row:
        bin_array_QCD[i] = col
        i+=1

In [ ]:
bin_array_QCD_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['QCD_bEnriched_HT'][''].errors:
    for col in row:
        bin_array_QCD_errors[i] = col
        i+=1

In [ ]:
bin_array_other = np.ones(40)
i=0
for row in MT_vs_sdmass['other'][''].counts:
    for col in row:
        bin_array_other[i] = col
        i+=1

In [ ]:
unrolled = [bin_array_Z,  bin_array_QCD, bin_array_tt, bin_array_W,bin_array_other,]
unrolled_errors = [bin_array_Z_errors, bin_array_QCD_errors, bin_array_tt_errors, bin_array_W_errors]

labels =[r'$Z\to\nu\nu$', r'$Multijet$', r'$t\bar{t}/single-t$', r'$W+jets$', r'$SM Higgs$',]

colors =['#00C239', '#93CCEA',  '#FE3F6B',  '#FED23F', '#EAB193']

fig, (ax) = plt.subplots(figsize=(10,10))
hep.cms.label(
    'Preliminary',
    loc=0,
    ax=ax,
    lumi = 36,
    #rlabel = '(14 TeV)',
    )

hep.histplot(
    [bin_array for bin_array in unrolled],
    #w2=[bin_array**2 for bin_array in unrolled_errors],
    histtype="fill",
    #density = True,
    stack=True,
    label=[label for label in labels],
    color=[color for color in colors],
    ax=ax
    )

hep.histplot(
    bin_array_signal,
    w2=bin_array_signal_errors**2,
    histtype="step",
    #density = True,
    stack=False,
    label=r'$m_A=1000, m_a=250$',
    #color=[colors[sample] for sample in backgrounds],
    ax=ax
    )

ax.set_xlabel(r'$Bin$')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
ax.legend(prop={'size': 9})

fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/unrolled_stack_1000_250_2.png')

In [ ]:
bin_array_signal2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_2000_750_central.counts[0])):
    for j in range(len(MT_vs_sdmass_2000_750_central.counts[:,i:i+1])):
        bin_array_signal2[k] = MT_vs_sdmass_2000_750_central.counts[j][i]
        k+=1

In [ ]:
bin_array_signal_errors2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_2000_750_central.errors[0])):
    for j in range(len(MT_vs_sdmass_2000_750_central.errors[:,i:i+1])):
        bin_array_signal_errors2[k] = MT_vs_sdmass_2000_750_central.errors[j][i]
        k+=1

In [ ]:
bin_array_Z2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_Z_central.counts[0])):
    for j in range(len(MT_vs_sdmass_Z_central.counts[:,i:i+1])):
        bin_array_Z2[k] = MT_vs_sdmass_Z_central.counts[j][i]
        k+=1

In [ ]:
bin_array_tt2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_tt_central.counts[0])):
    for j in range(len(MT_vs_sdmass_tt_central.counts[:,i:i+1])):
        bin_array_tt2[k] = MT_vs_sdmass_tt_central.counts[j][i]
        k+=1

In [ ]:
bin_array_W2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_W_central.counts[0])):
    for j in range(len(MT_vs_sdmass_W_central.counts[:,i:i+1])):
        bin_array_W2[k] = MT_vs_sdmass_W_central.counts[j][i]
        k+=1

In [ ]:
bin_array_QCD2 = np.ones(40)
i=0
k=0
for i in range(len(MT_vs_sdmass_QCD_central.counts[0])):
    for j in range(len(MT_vs_sdmass_QCD_central.counts[:,i:i+1])):
        bin_array_QCD2[k] = MT_vs_sdmass_QCD_central.counts[j][i]
        k+=1

In [ ]:
unrolled = [bin_array_Z2,  bin_array_QCD2, bin_array_tt2, bin_array_W2,]
#unrolled_errors = [bin_array_Z_errors, bin_array_QCD_errors, bin_array_tt_errors, bin_array_W_errors]

labels =[r'$ZJets\to\nu\nu$', r'$QCD\ b-enriched$', r'$t\bar{t}$', r'$WJets\to L\nu$', ]

colors =['#7DDC61', '#C061DC',  '#FE3F6B',  '#FED23F', ]

fig, (ax) = plt.subplots(figsize=(10,10))
hep.cms.label(
    'Preliminary',
    loc=0,
    ax=ax,
    #lumi = 3000,
    rlabel = '(14 TeV)',
    )

hep.histplot(
    [bin_array for bin_array in unrolled],
    #w2=[bin_array**2 for bin_array in unrolled_errors],
    histtype="fill",
    #density = True,
    stack=True,
    label=[label for label in labels],
    color=[color for color in colors],
    ax=ax
    )

hep.histplot(
    bin_array_signal2,
    w2=bin_array_signal_errors2**2,
    histtype="step",
    #density = True,
    stack=False,
    label='2HDMa_1500_150_10',
    #color=[colors[sample] for sample in backgrounds],
    ax=ax
    )

ax.set_xlabel(r'$Bin$')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
ax.legend(prop={'size': 14})

fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/unrolled_stack_2000_750_alt.png')